In [1]:
!pip install transformers datasets colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


# Inference Segmentation only

In [3]:
from transformers import AutoTokenizer, XLMRobertaForTokenClassification, pipeline
from tokenizers.pre_tokenizers import Whitespace

In [4]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
pre_tokenizer = Whitespace()

In [5]:
model = XLMRobertaForTokenClassification.from_pretrained('MrPotato/reference-segmentation-xlm-roberta-geocite')
model.to(device)

Some weights of the model checkpoint at MrPotato/reference-segmentation-xlm-roberta-geocite were not used when initializing XLMRobertaForTokenClassification: ['classifier_first.weight', 'classifier_first.bias', 'classifier_second.bias', 'classifier_second.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at MrPotato/reference-segmentation-xlm-roberta-geocite and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probab

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0): XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)

In [6]:
#ws_tkn = Whitespace()

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first")

In [7]:
# s1 = "Veröffentlicht von Bernt , D. in 1996 : Berg und Tal - Strukturprobleme im Tourismus, Raum , 22 : 10 - 13."
# s2 = "Wimmer H. (1985), Zur Ausländerbeschäftigungspolitik in Österreich. In: Wimmer H. (Hrsg.), Ausländische Arbeitskräfte in Österreich, S. 5-33. Frankfurt am Main."
# s3 = 'D E GEER, E. H. (1953): La varve Zero et les drainages successifs finaux du grand lac de barrage Central du Jämtland. - Cahiers Geologiques 20, S. 171-184, Thoiry. - - (1957): Old an new datings of Swedish ice lakes. - GFF 79, S. 93-100.'
# s4 = 'Kayser, K., W. Manshabd, H. Mensching, J. H. Schultze: Das Afrika-Kartenwerk. Ein Schwerpunktprogramm der Deutschen Forschungsgemeinschaft. Die Erde 97 (1966), S. 85—95.'
# s5 = 'H. Flohn, In München auf und Struktur des ostasiatischen Sommermonsuns. Vortrag auf der Geographentagung 1948‚ im Druck (mit Karten, Diagrammen und zahlreichen Literaturangaben).'
# s6 = 'Verband Deutscher Papierfabriken: Papier 83, Leistungsbericht der Papierindustrie, Bonn 1983.'
s1 = """
Vgl. von regierungsamtlicher Seite z.B. Bundesministerium
für Umwelt, Naturschutz und Reaktorsicherheit
(Hrsg.): Umweltpolitik. Eckwerte
der ökologischen Sanierung und Entwicklung in
den neuen Ländern. - Bonn 1990; von kritischen
Wissenschaftlern z.B. Petschow, U.: Entwicklungstrends
der Umweltsituation in den
neuen Bundesländern. Der wirtschaftliche Zusammenbruch
- Konflikt zwischen Arbeit und
Umwelt. In: Informationsdienst des Instituts für
Ökologische Wirtschaftsforschung GmbH
(IOW) und der Vereinigung für Okologische
Wirtschaftsforschung e.V. (VOW) (1991) 3-4,
S. 13-14; oder: Stratmann-Mertens, E.; Roske,
N.: Neuaufbau ohne Vorbild. Alternativen für
den ökonomischen Aufbau und Ecksteine für
den ökologischen Umbau in den neuen Bundesländern.
Gutachten im Auftrag des Bundesvorstandes
der GRUNEN. - Essen 1992
"""

In [8]:
#tkn_list, offsets = zip(*ws_tkn.pre_tokenize_str(s1))
#s = ' '.join(list(tkn_list))
nlp(s1)

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/token_classification.py:319: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn("Tokenizer does not support real words, using fallback heuristic", UserWarning)


[{'entity_group': 'LABEL_1',
  'score': 0.6254543,
  'word': 'Vgl.vonregierungsamtlicherSeitez.B.',
  'start': 1,
  'end': 40},
 {'entity_group': 'LABEL_0',
  'score': 0.5651532,
  'word': 'Bundesministerium für',
  'start': 41,
  'end': 62},
 {'entity_group': 'LABEL_1',
  'score': 0.64070636,
  'word': 'Umwelt,NaturschutzundReaktorsicherheit (Hrsg.):Umweltpolitik.Eckwerte derökologischenSanierungundEntwicklungin denneuenLändern.-',
  'start': 63,
  'end': 204},
 {'entity_group': 'LABEL_0',
  'score': 0.7012265,
  'word': 'Bonn',
  'start': 205,
  'end': 209},
 {'entity_group': 'LABEL_1',
  'score': 0.67450076,
  'word': '1990;vonkritischen Wissenschaftlernz.B.Petschow,',
  'start': 210,
  'end': 262},
 {'entity_group': 'LABEL_0',
  'score': 0.53430045,
  'word': 'U.:',
  'start': 263,
  'end': 266},
 {'entity_group': 'LABEL_1',
  'score': 0.6268028,
  'word': 'Entwicklungstrends derUmweltsituationinden neuenBundesländern.DerwirtschaftlicheZusammenbruch -KonfliktzwischenArbeitund Umwel

In [ ]:
from tokenizers.pre_tokenizers import Whitespace
from transformers import pipeline

sentence = "Veröffentlicht von Bernt , D. in 1996 : Berg und Tal - Strukturprobleme im Tourismus, Raum , 22 : 10 - 13."
sentence2 = "Wimmer H. (1985), Zur Ausländerbeschäftigungspolitik in Österreich. In: Wimmer H. (Hrsg.), Ausländische Arbeitskräfte in Österreich, S. 5-33. Frankfurt am Main."

tk = Whitespace()
tkn_list, offsets = zip(*tk.pre_tokenize_str(sentence2))
inputs = tokenizer(list(tkn_list),
                    is_split_into_words=True, 
                    return_offsets_mapping=True, 
                    padding='max_length', 
                    truncation=True, 
                    max_length=MAX_LEN,
                    return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, attention_mask=mask)
logits = outputs[0]

predicted_label_classes = logits.argmax(-1)
#print(predicted_label_classes) # shape (batch_size * seq_len, num_labels)
#flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

predicted_labels = [model.config.id2label[id] for id in predicted_label_classes.squeeze().tolist()]
#print(predicted_labels)

for id, label in zip(inputs.input_ids.squeeze().tolist(), predicted_labels):
  print(tokenizer.decode([id]), label)




tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

#for id, label in zip(encoding.input_ids.squeeze().tolist(), predicted_labels):
#  print(tokenizer.decode([id]), label)

prediction = []
for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
    print(token_pred, mapping)
  #only predictions on first word pieces are important
    if mapping[0] == 0 and mapping[1] != 0:
     prediction.append(token_pred[1])
    else:
     continue

#list(zip(list(tkn_list), prediction))

<s> O
Wi B-author
mmer I-author
H I-author
 I-author
. I-author
( O
1985 B-year
 O
), O
Zur B-title
Aus I-title
länder I-title
be I-title
schä I-title
ft I-title
igung I-title
s I-title
politik I-title
in I-title
Österreich I-title
 O
. O
In O
: O
Wi B-editor
mmer I-editor
H I-editor
 I-editor
. I-editor
( O
Hr O
s O
g O
 O
. O
), O
Aus B-source
ländische I-source
Arbeits I-source
kräfte I-source
in I-source
Österreich I-source
 O
, O
S O
 O
. O
5 B-fpage
- O
33 B-lpage
 O
. O
Frankfurt B-other
am I-other
Main I-other
 O
. O
</s> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> O
<pad> 

# Inference Segmentation and Blocking

## Imports

In [9]:
from transformers import AutoTokenizer, XLMRobertaForTokenClassification, Pipeline, AutoModelForTokenClassification, AutoModel, XLMRobertaTokenizerFast
from tokenizers.pre_tokenizers import Whitespace
from transformers.pipelines import PIPELINE_REGISTRY
from itertools import chain
from colorama import Fore, Back
from colorama import Style
import numpy as np

In [ ]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
tokenizer.pre_tokenizer = Whitespace()

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [11]:
labels = [
    'publisher', 'source', 'url', 'other', 'author', 'editor', 'lpage',
    'volume', 'year', 'issue', 'title', 'fpage', 'edition'
]
iob_labels = list(chain.from_iterable([['B-' + x, 'I-' + x] for x in labels])) + ['O']
id2label = {k:v for k,v in enumerate(iob_labels)}
label2id = {v:k for k,v in enumerate(iob_labels)}
id2label_ref = {k:v for k,v in enumerate(['B-ref', 'I-ref', ])}
label_ref2id = {v:k for k,v in enumerate(['B-ref', 'I-ref', ])}

In [12]:
len(iob_labels)

27

In [13]:
from transformers.models.xlm_roberta import XLMRobertaPreTrainedModel, XLMRobertaModel
from transformers.models.roberta import RobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers import PretrainedConfig
import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from typing import List, Optional, Tuple, Union


class XLMRobertaRefSegConfig(PretrainedConfig):
    r"""
    This is the configuration class to store the configuration of a [`XLMRobertaModel`] or a [`TFXLMRobertaModel`]. It
    is used to instantiate a XLM-RoBERTa model according to the specified arguments, defining the model architecture.
    Instantiating a configuration with the defaults will yield a similar configuration to that of the XLMRoBERTa
    [xlm-roberta-base](https://huggingface.co/xlm-roberta-base) architecture.
    Configuration objects inherit from [`PretrainedConfig`] and can be used to control the model outputs. Read the
    documentation from [`PretrainedConfig`] for more information.
    Args:
        vocab_size (`int`, *optional*, defaults to 30522):
            Vocabulary size of the XLM-RoBERTa model. Defines the number of different tokens that can be represented by
            the `inputs_ids` passed when calling [`XLMRobertaModel`] or [`TFXLMRobertaModel`].
        hidden_size (`int`, *optional*, defaults to 768):
            Dimensionality of the encoder layers and the pooler layer.
        num_hidden_layers (`int`, *optional*, defaults to 12):
            Number of hidden layers in the Transformer encoder.
        num_attention_heads (`int`, *optional*, defaults to 12):
            Number of attention heads for each attention layer in the Transformer encoder.
        intermediate_size (`int`, *optional*, defaults to 3072):
            Dimensionality of the "intermediate" (often named feed-forward) layer in the Transformer encoder.
        hidden_act (`str` or `Callable`, *optional*, defaults to `"gelu"`):
            The non-linear activation function (function or string) in the encoder and pooler. If string, `"gelu"`,
            `"relu"`, `"silu"` and `"gelu_new"` are supported.
        hidden_dropout_prob (`float`, *optional*, defaults to 0.1):
            The dropout probability for all fully connected layers in the embeddings, encoder, and pooler.
        attention_probs_dropout_prob (`float`, *optional*, defaults to 0.1):
            The dropout ratio for the attention probabilities.
        max_position_embeddings (`int`, *optional*, defaults to 512):
            The maximum sequence length that this model might ever be used with. Typically set this to something large
            just in case (e.g., 512 or 1024 or 2048).
        type_vocab_size (`int`, *optional*, defaults to 2):
            The vocabulary size of the `token_type_ids` passed when calling [`XLMRobertaModel`] or
            [`TFXLMRobertaModel`].
        initializer_range (`float`, *optional*, defaults to 0.02):
            The standard deviation of the truncated_normal_initializer for initializing all weight matrices.
        layer_norm_eps (`float`, *optional*, defaults to 1e-12):
            The epsilon used by the layer normalization layers.
        position_embedding_type (`str`, *optional*, defaults to `"absolute"`):
            Type of position embedding. Choose one of `"absolute"`, `"relative_key"`, `"relative_key_query"`. For
            positional embeddings use `"absolute"`. For more information on `"relative_key"`, please refer to
            [Self-Attention with Relative Position Representations (Shaw et al.)](https://arxiv.org/abs/1803.02155).
            For more information on `"relative_key_query"`, please refer to *Method 4* in [Improve Transformer Models
            with Better Relative Position Embeddings (Huang et al.)](https://arxiv.org/abs/2009.13658).
        is_decoder (`bool`, *optional*, defaults to `False`):
            Whether the model is used as a decoder or not. If `False`, the model is used as an encoder.
        use_cache (`bool`, *optional*, defaults to `True`):
            Whether or not the model should return the last key/values attentions (not used by all models). Only
            relevant if `config.is_decoder=True`.
        classifier_dropout (`float`, *optional*):
            The dropout ratio for the classification head.
    Examples:
    ```python
    >>> from transformers import XLMRobertaConfig, XLMRobertaModel
    >>> # Initializing a XLM-RoBERTa xlm-roberta-base style configuration
    >>> configuration = XLMRobertaConfig()
    >>> # Initializing a model (with random weights) from the xlm-roberta-base style configuration
    >>> model = XLMRobertaModel(configuration)
    >>> # Accessing the model configuration
    >>> configuration = model.config
    ```"""
    model_type = "xlm-roberta"
    def __init__(
            self,
            vocab_size=250002,
            hidden_size=768,
            num_hidden_layers=12,
            num_attention_heads=12,
            intermediate_size=3072,
            hidden_act="gelu",
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.1,
            max_position_embeddings=514,
            type_vocab_size=1,
            initializer_range=0.02,
            layer_norm_eps=1e-12,
            pad_token_id=1,
            bos_token_id=0,
            eos_token_id=2,
            position_embedding_type="absolute",
            use_cache=True,
            classifier_dropout=None,
            num_labels_first=29,
            num_labels_second=2,
            alpha=1.0,
            **kwargs
        ):
            super().__init__(pad_token_id=pad_token_id, bos_token_id=bos_token_id, eos_token_id=eos_token_id, **kwargs)

            self.vocab_size = vocab_size
            self.hidden_size = hidden_size
            self.num_hidden_layers = num_hidden_layers
            self.num_attention_heads = num_attention_heads
            self.hidden_act = hidden_act
            self.intermediate_size = intermediate_size
            self.hidden_dropout_prob = hidden_dropout_prob
            self.attention_probs_dropout_prob = attention_probs_dropout_prob
            self.max_position_embeddings = max_position_embeddings
            self.type_vocab_size = type_vocab_size
            self.initializer_range = initializer_range
            self.layer_norm_eps = layer_norm_eps
            self.position_embedding_type = position_embedding_type
            self.use_cache = use_cache
            self.classifier_dropout = classifier_dropout
            self.num_labels_first = num_labels_first
            self.num_labels_second = num_labels_second
            self.alpha = alpha
            super().__init__(pad_token_id=pad_token_id, bos_token_id=bos_token_id, eos_token_id=eos_token_id, **kwargs)

class XLMRobertaForReferenceSegmentation(XLMRobertaPreTrainedModel):
    _keys_to_ignore_on_load_unexpected = [r"pooler"]
    _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels_first = config.num_labels_first
        self.num_labels_second = config.num_labels_second
        self.alpha = config.alpha

        self.roberta = XLMRobertaModel(config, add_pooling_layer=False)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier_first = nn.Linear(config.hidden_size, self.num_labels_first)
        self.classifier_second = nn.Linear(config.hidden_size, self.num_labels_second)

        self.post_init()

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels_first: Optional[torch.LongTensor] = None,
        labels_second: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], TokenClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        sequence_output_first = self.dropout(sequence_output)
        logits_first = self.classifier_first(sequence_output_first)

        sequence_output_second = self.dropout(sequence_output)
        logits_second = self.classifier_second(sequence_output_second)

        loss = None
        if labels_first is not None and labels_second is not None:
            loss_fct_first = CrossEntropyLoss()
            loss_fct_second = CrossEntropyLoss()
            loss_first = loss_fct_first(logits_first.view(-1, self.num_labels_first), labels_first.view(-1))
            loss_second = loss_fct_second(logits_second.view(-1, self.num_labels_second), labels_second.view(-1))
            loss = loss_first + (self.alpha * loss_second)

        return TokenClassifierOutput(
            loss=loss,
            logits=[logits_first, logits_second],
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        ) 
         
        

In [17]:
config = XLMRobertaRefSegConfig(label2id)
model = XLMRobertaForReferenceSegmentation.from_pretrained('MrPotato/reference-segmentation-xlm-roberta-geocite', config=config)
model.to(device)

RuntimeError: ignored

In [14]:
style_map = {
    'publisher': f'{Back.LIGHTCYAN_EX}{Fore.BLACK}',
    'source': f'{Back.BLACK}{Fore.WHITE}',
    'url': f'{Back.LIGHTYELLOW_EX}{Fore.BLACK}',
    'other': f'{Back.BLUE}{Fore.WHITE}',
    'author': f'{Back.RED}{Fore.WHITE}',
    'editor': f'{Back.LIGHTRED_EX}{Fore.WHITE}',
    'lpage': f'{Back.LIGHTMAGENTA_EX}{Fore.WHITE}',
    'volume': f'{Back.YELLOW}{Fore.BLACK}',
    'year': f'{Back.CYAN}{Fore.WHITE}',
    'issue': f'{Back.LIGHTBLUE_EX}{Fore.WHITE}',
    'title': f'{Back.GREEN}{Fore.WHITE}',
    'fpage': f'{Back.LIGHTMAGENTA_EX}{Fore.WHITE}',
    'identifier': f'{Back.LIGHTBLACK_EX}{Fore.WHITE}',
    'edition': f'{Back.LIGHTYELLOW_EX}{Fore.BLACK}',
    'O': f'{Style.RESET_ALL}'
}

In [15]:
s1 = """
Sperling, IV. (1970): Einige psychologische und pädagogische
Fragen der Einführung in das Kartenlesen
und -verstehen. - In: Hinrichs, E. u. a. (1970): Der Atlas
im Erdkundeunterricht. Der Erdkundeunterricht H. 11,
S. 41-50.
Sperling, IV. (1974): Karten lesen und Kartengebraucli
im Unterricht, eine Bibliographie. - In: Nachrichtenblatt
d. Vermessungs- und Katasterverwaltung Rhein
land-Pfalz 17 (Sonderheft).
Stückrath, F. (1955,1968): Kind und Raum - psychologische
Voraussetzungen der Raumlehre in der Volks
schule. München.
Vernon, M. D. (Hrsg. 1966,1970): Experiments in Visual
Perception. London (Penguin Books).
"""

s2 = """
Nasrin Mostafazadeh, Michael Roth, Annie Louis,
Nathanael Chambers, and James Allen. 2017. LSDSem 2017 shared task: The story cloze test. In
Proceedings of the 2nd Workshop on Linking Models of Lexical, Sentential and Discourse-level Semantics, pages 46–51, Valencia, Spain. Association for
Computational Linguistics.
Inbar Oren, Jonathan Herzig, Nitish Gupta, Matt Gardner, and Jonathan Berant. 2020. Improving compositional generalization in semantic parsing. In Findings of the Association for Computational Linguistics: EMNLP 2020, pages 2482–2495, Online. Association for Computational Linguistics.
Myle Ott, Sergey Edunov, Alexei Baevski, Angela
Fan, Sam Gross, Nathan Ng, David Grangier, and
Michael Auli. 2019. fairseq: A fast, extensible
toolkit for sequence modeling. In Proceedings of
the 2019 Conference of the North American Chapter of the Association for Computational Linguistics
(Demonstrations), pages 48–53, Minneapolis, Minnesota. Association for Computational Linguistics.
Kishore Papineni, Salim Roukos, Todd Ward, and WeiJing Zhu. 2002. Bleu: a method for automatic evaluation of machine translation. In Proceedings of
the 40th Annual Meeting of the Association for Computational Linguistics, pages 311–318, Philadelphia,
Pennsylvania, USA. Association for Computational
Linguistics.
Matthew E. Peters, Mark Neumann, Mohit Iyyer, Matt
Gardner, Christopher Clark, Kenton Lee, and Luke
Zettlemoyer. 2018. Deep contextualized word representations. In Proceedings of the 2018 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long Papers), pages
2227–2237, New Orleans, Louisiana. Association
for Computational Linguistics.
Linlu Qiu, Peter Shaw, Panupong Pasupat, Pawel
Nowak, Tal Linzen, Fei Sha, and Kristina Toutanova.
2022. Improving compositional generalization with
latent structure and data augmentation. In Proceedings of the 2022 Conference of the North American Chapter of the Association for Computational
Linguistics: Human Language Technologies, pages
4341–4362, Seattle, United States. Association for
Computational Linguistics.
"""

s3 = '3 Vgl. E. Augenreich, Erdkunde im Gesamtunterricht. Geogr. Rundschau +49.'

s4 = """
CARSTEN BUTSCH LITERATUR Baldwin-Edwards, M. (2011): Labour immigration and labour markets in the GCC countries: national patterns and trends. http://eprints.lse.
ac.uk/55239/1/Baldwin-Edwards_2011.pdt [Zugriff am 14.7.2014] BMI, Bundesministerium des Inneren, und BMF, Bundesamt für Migration und Flüchtlinge (2013):
ac.uk/55239/1/Baldwin-Edwards_2011.pdt [Zugriff am 14.7.2014] BMI, Bundesministerium des Inneren, und BMF, Bundesamt für Migration und Flüchtlinge (2013):
Migrationsbericht 2012. Berlin Deffner, V. (2014): Gelebte Translokalität von indischen Studierenden in Aa- chen. Geographische Rundschau 66 [11], 5. 18-23 Dickinson, J. und
A.J. Bailey (20071: (Relmembering diaspora: Uneven geo- graphies of Indian dual citizenship. Political Geography 26 [7], S. 757-774 Soel, U. (2002): Von Freiheitskämpfern zu
Computer-Indern. Südasiaten in Deutschland. Südasien 22 (1], S. 70-7 Gottschlich, P (20121: Carim-India Research Report 2012/03, German Case Study. San Domenico de
Fiseole, http://cadmus.eui.eu/hand- le/1814/20822 (Zugriff am 19.12.2012) Hunger, U. (2000): Vom „Brain-Drain” zum „Brain-Gain”. Migration, Netzwerk- bildung und
sozio-ökonomische Entwickiung: das Beispiel der indischen „Software-Migranten“. IMIS-Beiträge 16. 5. 7-22 Jain, P. (1982): Indians Abroad: A Current Population Estimate.
Economic and Political Weekly 17.18), 5. 299-304 Jain, P. [2011]: British Colonialism and International Migration from India Siliconindia (2012): 15 Nations Sending Highest
Remittances to India. www. siliconindia.com/news/business/15-Nations-Sending-Highest-Remit- tances-to-India-nid-147478-cid-3.html Singhvi, L.M. (2001: Report ofthe High
Level Committee on the Indian Dia- spora. Government of India, Ministry of External Affairs, Non-Resident Indian and Persons of Indian Origin Division. New Delhi Statistisches
Bundesamt (2005a]: Bevölkerung und Erwerbstätigkeit Aus- tändische Bevölkerung sowie Einbürgerungen. Wiesbaden Statistisches Bundesamt (20056): Bevölkerung und
Erwerbstätigkeit. Vor- täufige Wanderungsergebnisse. Wiesbaden Tinker, H. (1974): A New System of Slavery: The Export of indian Labour Overseas. Oxford Tinker, H. (1977):
The Banayn Tree: Overseas Emigrants form India, Pakistan and Bagladesh. Oxford Venier, P. (2011): Development of Entrepreneurial Initiatives in the UAE among Kerala
Emigrants. In: R. Irudaya und M. Percot [Hrsg.): Dynamics of Indian Migration. London, New York, New Delhi, 5. 164-194 Wortd Bank [2014]: Remittance Inflows.
http://siteresources.worldbank.org/ INTPROSPECTS/Resources/334934-1288990760745/RemittanceData_ Inflows_Apr2014.xls [Zugriff am 10.07.2014) Four Destinations. In:
R. Irudaya und M. Percot [Hrsg.]: Dynamics of Indian Migration. London, New York, New Dalhi, 5. 23-48 Jayram, N. [2004]: Introduction: The Study of Indian Diaspora. In: N.
Jayram (Hrsg.]: The Indian Dispora. Dynamics of Migration. New Delhi, 5. 15-43 Khadrıa, B. (1999). The Migration of Knowledge Workers: Second-Generati- on Effects of India’s
Brain Drain. New Delhi MOIA, Ministry of Overseas Indian Affairs [2012]: Population of Nonresident Indians Countrywise. http://moia.gov.in/writereaddata/pdf/NRISPIOS-
Datal15-08-12)new.pdf [Zugriff am 12.6.2014) Oonk, 6. (2007]: Global Indian Diasporas: Exploring Trajectories of Migration and Theory. In: G. Oonk [Hrsg.): Global Indian
diasporas: Exploring trajectories of migration and theory. Amsterdam, S. 9-27 Dr. CARSTEN BUTSCH, geb. 1980 Geographisches Institut der Universität zu Köln,
1948 in Hildesheim geboren. Seine akademische Ausbildung erfuhr er an der Universität Göttingen, wo er von 1968 bis 1974 die Fächer Anglistik und Geographie studierte. Sein
Gesellschaft im Cauvery-Delta (Südindien)“, mit der H.-G. Bohle 1979 promoviert wurde. Am Beispiel des Dorfes Sirangarajapuram (Tamil Nadu) zeigte Bohle die Folgen der
Welt“. Nach einer einjährigen Tätigkeit als Privatdozent an der Göttinger Uni- versität erfolgte 1986 der Ruf aufeine C2-Professur am Institut für Kulturgeo- graphie der
(1989-1995) baute H.-G. Bohle die von Manshard 46 propagierte Entwicklungsländergeographie zur sozialwissenschaftlichen Geographischen Entwicklungsforschung aus.
Vulnerable Food Systems belegt. Von 1995 bis 2004 war H.-G. Bohle als Geographie-Direktor am Südasien-Institut der Universität Heidelberg tätig, wo er den Grundstein für die
moderne Geographische Risikoforschung legte.
"""

s5 = """
Chandler, T. J.: Urban climatology — Inventory and prospect/
Summary and conclusions of the symposium; in:
WMO, T. N 108. S. 1-14, 375-377. 1970,
Eriksen, W.: Beiträge zum Stadtklima von Kiel; Schr.
Geogr. Inst. Univ. Kiel. 22,1. 1964.
Ders.: Probleme der Stadt- und Geländeklimatologie; Erträge
der Forschung. Bd. 35. Darmstadt 1975.
Fezer, F.: Lokalklimatologische Interpretation von Thermalluftbildern;
in: Bildmessung und Luftbildwesen H. 4.
S. 152-158. 1975.
Flohn, H.: Klimaschwankung oder Klima-Modifikation?
in: Arb. z. Allg. Klimatologie. S. 291-309. Darmstadt
1971.
Jauregui, E.: Untersuchungen zum Stadtklima von Mexiko-
Stadt; Diss. Bonn 1973.
Kessler, A.: Über den Tagesgang von Oberflächentemperaturen
in der Bonner Innenstadt an einem sommerlichen
Strahlungstag; in: Erdkunde, 25,1. S. 13-20. 1971.
Kratzer, P. A.: Das Stadtklima. Die Wissenschaft, Bd. 90.
2. Aufl., Braunschweig 1956.
Landsberg, H. E.: The urban area as target for meteorological
research; in: Bonner Met. Abh. 17. S. 475-480. 1974.
Lindgvist, S.: Bebyggelseklimatiska Studier;, Medd. f.
Lunds Univ. Geogr. Inst. Avh. LXI. 1970.
Miess, M.: Planungsrelevante und kausalanalytische Aspekte
der Stadtklimatologie; in: Landschaft und Stadt, 6,1.
S.9 ff. 1974.
Oke, T. R.: Towards a more rational understanding of the
urban heat island; in: Climat. Bull. 5. S. 1-20. Montreal
1969.
Oke, T. R. und F. G. Hannell: The form of the urban heat
island in Hamilton, Canada; in: WMO, T.N. 108,
S. 113-126. 1970.
Regionale Planungsgemeinschaft Untermain: Lufthygienisch-
meteorologische Modelluntersuchung in der Region
Untermain. 5. Arbeitsbericht; Frankfurt/Main 1974.
Terjung, H. W. und St. S. F. Louie, B. Birkeneder and Christian Haase Solar radiation and
urban heat islands; in: Ann. of Ass. Am. Geogr. 63.2.
S. 181-207. 1973.
"""

sentence = s5

In [16]:
print('Legend:')
for k, v in style_map.items():
    print(v+k)
print()
inputs = tokenizer(sentence,
                    #is_split_into_words=True, 
                    return_offsets_mapping=True, 
                    padding='max_length', 
                    truncation=True, 
                    max_length=512,
                    return_tensors="pt",
                    return_overflowing_tokens=True )

# move to gpu
ids = inputs["input_ids"].to(device)
masks = inputs["attention_mask"].to(device)
offsets = inputs["offset_mapping"].to(device).squeeze().tolist()
# forward pass

#for id, mask, offset in zip(ids, masks, offsets):
outputs = model(ids, attention_mask=masks)

logits_seg = outputs[0][0]
logits_ref = outputs[0][1]
segmented_parts = []
segments = []
for i, (seg, ref) in enumerate(zip(logits_seg, logits_ref)):

    predicted_label_classes_seg= seg.argmax(-1)
    predicted_label_classes_ref = ref.argmax(-1)
    #print(predicted_label_classes) # shape (batch_size * seq_len, num_labels)
    #flattened_predictions = torch.argmax(logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    predicted_labels_seg = [id2label[id] for id in predicted_label_classes_seg.squeeze().tolist()]
    predicted_labels_ref = [id2label_ref[id] for id in predicted_label_classes_ref.squeeze().tolist()]
    #print(predicted_labels)

    
    
    for id, label_seg, label_ref, off in zip(inputs.input_ids[i].squeeze().tolist(), predicted_labels_seg, predicted_labels_ref, offsets[i]):
        tk = tokenizer.decode([id])
        if tk not in tokenizer.special_tokens_map.values():
            if label_ref == 'B-ref':
                if segments:
                    segmented_parts.append(segments)
                    segments = []
            
            segments.append((tk, label_seg, off))
segmented_parts.append(segments)


print(f'{len(segmented_parts)} references found:')

for p in segmented_parts:
    s = f''
    offset_last = None
    for tk, tag, offset in p:
        if offset_last != offset[0]:
            s += ' '
        if tag.startswith('B'):
            s += style_map[tag[2:]]
        elif tag == 'O':
            s += style_map[tag]
        s += tk
        offset_last = offset[1]
    print(s)
print('####################\n')

Legend:
publisher
source
url
other
author
editor
lpage
volume
year
issue
title
fpage
identifier
edition
O



TypeError: ignored

In [ ]:
class RefSegPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, sentence, offset_mapping=None):
        truncation = True if self.tokenizer.model_max_length and self.tokenizer.model_max_length > 0 else False
        model_inputs = self.tokenizer(
            sentence,
            return_offsets_mapping=True, 
            padding='max_length', 
            truncation=True, 
            max_length=512,
            return_tensors="pt",
            return_special_tokens_mask=True,
            return_overflowing_tokens=True
        )

        if offset_mapping:
            model_inputs["offset_mapping"] = offset_mapping

        model_inputs["sentence"] = sentence

        return model_inputs

    def _forward(self, model_inputs):
        special_tokens_mask = model_inputs.pop("special_tokens_mask")
        offset_mapping = model_inputs.pop("offset_mapping", None)
        sentence = model_inputs.pop("sentence")
        overflow_mapping = model_inputs.pop("overflow_to_sample_mapping")
        if self.framework == "tf":
            logits = self.model(model_inputs.data)[0]
        else:
            logits = self.model(**model_inputs)[0]

        return {
            "logits": logits,
            "special_tokens_mask": special_tokens_mask,
            "offset_mapping": offset_mapping,
            "overflow_mapping": overflow_mapping,
            "sentence": sentence,
            **model_inputs,
        }

    def postprocess(self, model_outputs):
        # if ignore_labels is None:
        ignore_labels = ["O"]
        logits_seg = model_outputs["logits"][0].numpy()
        logits_ref = model_outputs["logits"][1].numpy()
        sentence = model_outputs["sentence"]
        input_ids = model_outputs["input_ids"]
        special_tokens_mask = model_outputs["special_tokens_mask"]
        overflow_mapping = model_outputs["overflow_mapping"]
        
        offset_mapping = model_outputs["offset_mapping"] if model_outputs["offset_mapping"] is not None else None

        maxes_seg = np.max(logits_seg, axis=-1, keepdims=True)
        shifted_exp_seg = np.exp(logits_seg - maxes_seg)
        scores_seg = shifted_exp_seg / shifted_exp_seg.sum(axis=-1, keepdims=True)

        maxes_ref = np.max(logits_ref, axis=-1, keepdims=True)
        shifted_exp_ref = np.exp(logits_ref - maxes_ref)
        scores_ref = shifted_exp_ref / shifted_exp_ref.sum(axis=-1, keepdims=True)

        pre_entities = self.gather_pre_entities(
            sentence, input_ids, scores_seg, scores_ref, offset_mapping, special_tokens_mask
        )
        grouped_entities = self.aggregate(pre_entities)


        cleaned_groups = []
        for group in grouped_entities:
            entities = [
                entity
                for entity in group
                if entity.get("entity_group", None) not in ignore_labels
            ]
            cleaned_groups.append(entities)
        return {
            "number_of_references": len(cleaned_groups),
            "references": cleaned_groups,
            }

    def gather_pre_entities(
        self,
        sentence: str,
        input_ids: np.ndarray,
        scores_seg: np.ndarray,
        scores_ref: np.ndarray,
        offset_mappings: Optional[List[Tuple[int, int]]],
        special_tokens_masks: np.ndarray,
    ) -> List[dict]:
        """Fuse various numpy arrays into dicts with all the information needed for aggregation"""
        pre_entities = []
        for idx_list, (input_id, offset_mapping, special_tokens_mask, s_seg, s_ref) in enumerate(zip(input_ids, offset_mappings, special_tokens_masks, scores_seg, scores_ref)):
            for idx, iid in enumerate(input_id):

                if special_tokens_mask[idx]:
                    continue

                word = self.tokenizer.convert_ids_to_tokens(int(input_id[idx]))
                if offset_mapping is not None:
                    start_ind, end_ind = offset_mapping[idx]
                    if not isinstance(start_ind, int):
                        if self.framework == "pt":
                            start_ind = start_ind.item()
                            end_ind = end_ind.item()
                    word_ref = sentence[start_ind:end_ind]
                    if getattr(self.tokenizer._tokenizer.model, "continuing_subword_prefix", None):
                        is_subword = len(word) != len(word_ref)
                    else:
                        is_subword = len(word) == len(word_ref)

                    if int(input_id[idx]) == self.tokenizer.unk_token_id:
                        word = word_ref
                        is_subword = False
                else:
                    start_ind = None
                    end_ind = None
                    is_subword = False

                pre_entity = {
                    "word": word,
                    "scores_seg": s_seg[idx],
                    "scores_ref": s_ref[idx],
                    "start": start_ind,
                    "end": end_ind,
                    "index": idx,
                    "is_subword": is_subword,
                }
                pre_entities.append(pre_entity)
        return pre_entities
        

    def aggregate(self, pre_entities: List[dict]) -> List[dict]:
        entities = self.aggregate_words(pre_entities)

        return self.group_entities(entities)


    def aggregate_word(self, entities: List[dict]) -> dict:
        word = self.tokenizer.convert_tokens_to_string([entity["word"] for entity in entities])
        scores_seg = entities[0]["scores_seg"]
        idx_seg = scores_seg.argmax()
        score_seg = scores_seg[idx_seg]
        entity_seg = id2label[idx_seg]

        scores_ref = np.stack([entity["scores_ref"] for entity in entities])
        indices_ref = scores_ref.argmax(axis=1)
        idx_ref = 1 if all(indices_ref) else 0
        #score_ref = 1
        entity_ref = id2label_ref[idx_ref]

        new_entity = {
            "entity_seg": entity_seg,
            "score_seg": score_seg,
            "entity_ref": entity_ref,
            #"score_ref": score_ref,
            "word": word,
            "start": entities[0]["start"],
            "end": entities[-1]["end"],
        }
        return new_entity


    def aggregate_words(self, entities: List[dict]) -> List[dict]:
        """
        Override tokens from a given word that disagree to force agreement on word boundaries.
        Example: micro|soft| com|pany| B-ENT I-NAME I-ENT I-ENT will be rewritten with first strategy as microsoft|
        company| B-ENT I-ENT
        """
        word_entities = []
        word_group = None
        for entity in entities:
            if word_group is None:
                word_group = [entity]
            elif entity["is_subword"]:
                word_group.append(entity)
            else:
                word_entities.append(self.aggregate_word(word_group))
                word_group = [entity]
        word_entities.append(self.aggregate_word(word_group))
        return word_entities


    def group_entities(self, entities: List[dict]) -> List[dict]:
        """
        Find and group together the adjacent tokens with the same entity predicted.
        Args:
            entities (`dict`): The entities predicted by the pipeline.
        """
        entity_chunk = []
        entity_chunk_disagg = []
        
        for entity in entities:
            if not entity_chunk_disagg:
                entity_chunk_disagg.append(entity)
                continue

            bi_ref, tag_ref = self.get_tag(entity["entity_ref"])
            last_bi_ref, last_tag_ref = self.get_tag(entity_chunk_disagg[-1]["entity_ref"])

            if tag_ref == last_tag_ref and bi_ref != "B":
                entity_chunk_disagg.append(entity)
            else:
                entity_chunk.append(entity_chunk_disagg)
                entity_chunk_disagg = [entity]

        if entity_chunk_disagg:
            entity_chunk.append(entity_chunk_disagg)

        entity_chunks_all = []

        for chunk in entity_chunk:
        
            entity_groups = []
            entity_group_disagg = []

            for entity in chunk:
                if not entity_group_disagg:
                    entity_group_disagg.append(entity)
                    continue

                bi_seg, tag_seg = self.get_tag(entity["entity_seg"])
                last_bi_seg, last_tag_seg = self.get_tag(entity_group_disagg[-1]["entity_seg"])

                if tag_seg == last_tag_seg and bi_seg != "B":
                    entity_group_disagg.append(entity)
                else:
                    entity_groups.append(self.group_sub_entities(entity_group_disagg))
                    entity_group_disagg = [entity]

            if entity_group_disagg:
                entity_groups.append(self.group_sub_entities(entity_group_disagg))

            entity_chunks_all.append(entity_groups)

        return entity_chunks_all


    def group_sub_entities(self, entities: List[dict]) -> dict:
        """
        Group together the adjacent tokens with the same entity predicted.
        Args:
            entities (`dict`): The entities predicted by the pipeline.
        """
        entity = entities[0]["entity_seg"].split("-")[-1]
        scores = np.nanmean([entity["score_seg"] for entity in entities])
        tokens = [entity["word"] for entity in entities]

        entity_group = {
            "entity_group": entity,
            "score": np.mean(scores),
            "word": " ".join(tokens),
            "start": entities[0]["start"],
            "end": entities[-1]["end"],
        }
        return entity_group


    def get_tag(self, entity_name: str) -> Tuple[str, str]:
        if entity_name.startswith("B-"):
            bi = "B"
            tag = entity_name[2:]
        elif entity_name.startswith("I-"):
            bi = "I"
            tag = entity_name[2:]
        else:
            bi = "I"
            tag = entity_name
        return bi, tag


In [ ]:
PIPELINE_REGISTRY.register_pipeline(
    "ref-seg",
    pipeline_class=RefSegPipeline,
    pt_model=AutoModelForTokenClassification,
    default={"pt": ("MrPotato/ReferenceSegmentationGER_blocking")},
    type="text",  # current support type: text, audio, image, multimodal
)

In [ ]:
from transformers import pipeline

classifier = pipeline("ref-seg", model=model, tokenizer=tokenizer)

In [ ]:
output = classifier(s4)

In [ ]:
for o in output['references']:
    print('############################################')
    print(o)

############################################
[{'entity_group': 'editor', 'score': 0.29919195, 'word': 'CARSTEN BUTSCH', 'start': 1, 'end': 15}, {'entity_group': 'publisher', 'score': 0.35312995, 'word': 'LITERATUR', 'start': 16, 'end': 25}]
############################################
[{'entity_group': 'author', 'score': 0.99133, 'word': 'Baldwin-Edwards, M.', 'start': 26, 'end': 45}, {'entity_group': 'year', 'score': 0.98741794, 'word': '(2011):', 'start': 46, 'end': 53}, {'entity_group': 'title', 'score': 0.9966536, 'word': 'Labour immigration and labour markets in the GCC countries: national patterns and trends.', 'start': 54, 'end': 143}, {'entity_group': 'url', 'score': 0.89264, 'word': 'http://eprints.lse. ac.uk/55239/1/Baldwin-Edwards_2011.pdt', 'start': 144, 'end': 202}]
############################################
[{'entity_group': 'editor', 'score': 0.9613292, 'word': 'BMI, Bundesministerium des Inneren, und BMF, Bundesamt für Migration und Flüchtlinge', 'start': 226, 'end': 

In [ ]:
output

{'number_of_references': 4,
 'references': [[{'entity_group': 'author',
    'score': 0.9782381,
    'word': 'Sperling, IV.',
    'start': 1,
    'end': 14},
   {'entity_group': 'title',
    'score': 0.9971077,
    'word': 'Einige psychologische und pädagogische Fragen der Einführung in das Kartenlesen und -verstehen.',
    'start': 23,
    'end': 118},
   {'entity_group': 'editor',
    'score': 0.97274506,
    'word': 'Hinrichs, E.',
    'start': 125,
    'end': 137},
   {'entity_group': 'other',
    'score': 0.6808764,
    'word': 'u. a.',
    'start': 138,
    'end': 143},
   {'entity_group': 'source',
    'score': 0.9323127,
    'word': 'Der Atlas im Erdkundeunterricht.',
    'start': 152,
    'end': 184},
   {'entity_group': 'source',
    'score': 0.8969821,
    'word': 'Der Erdkundeunterricht',
    'start': 185,
    'end': 207},
   {'entity_group': 'volume',
    'score': 0.9741737,
    'word': '11,',
    'start': 211,
    'end': 214},
   {'entity_group': 'fpage',
    'score': 0.98

In [ ]:
from transformers import pipeline

classifier = pipeline(model="MrPotato/ref-seg-ger", tokenizer=tokenizer, trust_remote_code=True)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Downloading:   0%|          | 0.00/6.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Downloading:   0%|          | 0.00/3.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
classifier(s5)

{'number_of_references': 15,
 'references': [[{'entity_group': 'author',
    'score': 0.9897008,
    'word': 'Chandler, T. J.:',
    'start': 1,
    'end': 17},
   {'entity_group': 'title',
    'score': 0.9891793,
    'word': 'Urban climatology — Inventory and prospect/ Summary and conclusions of the symposium;',
    'start': 18,
    'end': 103},
   {'entity_group': 'source',
    'score': 0.89688444,
    'word': 'WMO, T. N',
    'start': 108,
    'end': 117},
   {'entity_group': 'volume',
    'score': 0.95149845,
    'word': '108.',
    'start': 118,
    'end': 122},
   {'entity_group': 'issue',
    'score': 0.7299838,
    'word': '1-14,',
    'start': 126,
    'end': 131},
   {'entity_group': 'fpage',
    'score': 0.9856747,
    'word': '375-377.',
    'start': 132,
    'end': 140},
   {'entity_group': 'year',
    'score': 0.98526263,
    'word': '1970,',
    'start': 141,
    'end': 146}],
  [{'entity_group': 'author',
    'score': 0.98873806,
    'word': 'Eriksen, W.:',
    'start':

In [ ]:
shifted_exp.shape

torch.Size([3, 512, 29])

In [ ]:
scores = shifted_exp / shifted_exp.sum(axis=-1, keepdims=True)

In [ ]:
np.argmax(scores[0][1])

tensor(10)

In [ ]:
from transformers import AutoTokenizer, XLMRobertaForTokenClassification, pipeline
from tokenizers.pre_tokenizers import Whitespace
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("MrPotato/ReferenceSegmentation")
classifier = pipeline(model="MrPotato/ReferenceSegmentation", tokenizer=tokenizer, trust_remote_code=True)

Downloading:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
input = """
van Houtum, H. und T. van Naerssen
(2002): Bordering, Ordering and Othering.
In: Journal of Economic and Social
Geography 93, 2, S. 125-136.
Wastl-Walter, D. (Hrsg.) (2011): The
Ashgate Research Companion to Border
Studies. Farnham.
Wille, C. (2014): B/Ordering in der
Großregion. Mobilitäten - Grenzen -
Identitäten. In: Goulet, V. u. C. Vatter
(Hrsg.): Mediale Felder und Grenzen in
der Großregion SaarLorLux und in Europa.
Saarbrücken, S. 51-64.
"""

In [ ]:
sentence_copy = input

offset = 0
delimiter = '\n#####\n'
for group_list in classifier(sentence_copy)['references']:
    last_end = 0
    for group in group_list:
        sentence_copy = sentence_copy[:int(group['start'])+offset] + style_map[group['entity_group']] + sentence_copy[int(group['start'])+offset:]
        offset += len(style_map[group['entity_group']])
        sentence_copy = sentence_copy[:int(group['end'])+offset] + style_map['O'] + sentence_copy[int(group['end'])+offset:]
        offset += len(style_map['O'])
        last_end = int(group['end'])+offset
    sentence_copy = sentence_copy[:last_end] + delimiter + sentence_copy[last_end:]
    offset += len(delimiter)

In [ ]:
print(sentence_copy)


van Houtum, H. und T. van Naerssen
(2002): Bordering, Ordering and Othering.
In: Journal of Economic and Social
Geography 93, 2, S. 125-136.
#####

Wastl-Walter, D. (Hrsg.) (2011): The
Ashgate Research Companion to Border
Studies. Farnham.
#####

Wille, C. (2014): B/Ordering in der
Großregion. Mobilitäten - Grenzen -
Identitäten. In: Goulet, V. u. C. Vatter
(Hrsg.): Mediale Felder und Grenzen in
der Großregion SaarLorLux und in Europa.
Saarbrücken, S. 51-64.
#####




In [ ]:
classifier(input)

{'number_of_references': 3,
 'references': [[{'entity_group': 'author',
    'score': 0.9722815,
    'word': 'van Houtum, H.',
    'start': 1,
    'end': 15},
   {'entity_group': 'author',
    'score': 0.96468794,
    'word': 'T.',
    'start': 19,
    'end': 21},
   {'entity_group': 'author',
    'score': 0.8997587,
    'word': 'van Naerssen',
    'start': 22,
    'end': 34},
   {'entity_group': 'year',
    'score': 0.9905195,
    'word': '(2002):',
    'start': 35,
    'end': 42},
   {'entity_group': 'title',
    'score': 0.9941519,
    'word': 'Bordering, Ordering and Othering.',
    'start': 43,
    'end': 76},
   {'entity_group': 'source',
    'score': 0.98276806,
    'word': 'Journal of Economic and Social Geography',
    'start': 81,
    'end': 121},
   {'entity_group': 'volume',
    'score': 0.9904174,
    'word': '93,',
    'start': 122,
    'end': 125},
   {'entity_group': 'issue',
    'score': 0.9391241,
    'word': '2,',
    'start': 126,
    'end': 128},
   {'entity_group':